In [6]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume
from ctypes import cast,POINTER
import comtypes
import scipy.io.wavfile as wav
import scipy.signal as sig

In [7]:
%matplotlib tk

In [8]:
audioSample = 1024*4
Format = pyaudio.paInt16
channels = 2
rate = 44100

In [9]:
p = pyaudio.PyAudio()
stream = p.open(format = Format,
                channels=2,
                rate=rate,
                input=True,
                frames_per_buffer = audioSample,
                input_device_index = 2)
fig,(ax1,ax2) = plt.subplots(2,1)
x = np.arange(0,2*audioSample,2)
ax1.set_xlim(0,audioSample)
ax1.set_ylim(-1,1)
line, = ax1.plot(x,np.random.rand(audioSample))
line2, = ax1.plot(x,np.random.rand(audioSample),color='orange')
plt.ion()
plt.show()
NPERSEG = 1024*2
while True:
    data = stream.read(audioSample)
    dayta = np.frombuffer(data,dtype=np.int16)
    dayta = np.reshape(dayta, (audioSample, 2))
    line.set_ydata(dayta[:,0]/32767)
    line2.set_ydata(dayta[:,1]/32767)
    f,t,Sxx = sig.spectrogram(dayta[:,0],fs=rate,window = "hann",nperseg=NPERSEG)
    ax2.pcolormesh(t,f,10*np.log10(Sxx))
    fig.canvas.draw()
    fig.canvas.flush_events()
plt.show()

TclError: invalid command name "pyimage20"